## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-08-10-50-00 +0000,wsj,Eurozone’s Jobless Rate Unexpectedly Falls for...,https://www.wsj.com/economy/jobs/eurozones-job...
1,2026-01-08-10-49-52 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2026/01/08/world/...
2,2026-01-08-10-38-31 +0000,bbc,One million without heat and water after Russi...,https://www.bbc.com/news/articles/cgk8jv4r8nmo...
3,2026-01-08-10-37-25 +0000,bbc,Tesco and M&S report strong Christmas food sales,https://www.bbc.com/news/articles/cp82ylmy6d1o...
4,2026-01-08-10-33-14 +0000,bbc,Nigeria's ex-justice minister granted bail in ...,https://www.bbc.com/news/articles/c9vxmx4rgr0o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,67
82,venezuela,32
236,oil,26
168,new,21
89,shooting,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
342,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...,154
149,2026-01-08-00-40-05 +0000,nyt,U.S. Details Plan for Venezuela’s Oil Sales Af...,https://www.nytimes.com/2026/01/07/business/en...,153
58,2026-01-08-07-59-45 +0000,nyt,We Pressed Trump on His Conclusion About the I...,https://www.nytimes.com/2026/01/08/us/politics...,139
233,2026-01-07-21-17-14 +0000,nyt,"After Maduro Ouster, Trump’s Venezuela Moves P...",https://www.nytimes.com/2026/01/07/us/politics...,136
174,2026-01-07-23-24-04 +0000,nyt,"During Maduro Raid in Venezuela, a Close Call ...",https://www.nytimes.com/2026/01/07/us/politics...,135


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
342,154,2026-01-07-15-31-28 +0000,nyt,Trump Claims Venezuela Will Send U.S. Millions...,https://www.nytimes.com/2026/01/06/business/ve...
127,90,2026-01-08-02-04-21 +0000,nypost,Minneapolis Mayor Jacob Frey flips out on CNN ...,https://nypost.com/2026/01/07/us-news/minneapo...
136,64,2026-01-08-01-34-00 +0000,wsj,Trump Administration Draws Up New Legal Justif...,https://www.wsj.com/politics/policy/trump-admi...
359,48,2026-01-07-13-34-42 +0000,cbc,U.S. Secretary of State Rubio says he will mee...,https://www.cbc.ca/news/world/us-greenland-eur...
358,46,2026-01-07-13-45-59 +0000,cbc,U.S. seeks to assert its control over Venezuel...,https://www.cbc.ca/news/world/united-states-ru...
9,44,2026-01-08-10-30-00 +0000,wsj,Some mail won’t be postmarked the day you drop...,https://www.wsj.com/politics/policy/a-letter-w...
90,42,2026-01-08-04-25-00 +0000,wsj,"Rep. Steny Hoyer of Maryland, a longtime membe...",https://www.wsj.com/politics/policy/steny-hoye...
328,40,2026-01-07-16-22-02 +0000,bbc,Canada PM Mark Carney to visit China next week...,https://www.bbc.com/news/articles/cevnek9k32no...
141,40,2026-01-08-01-22-04 +0000,nypost,"Video appears to show hysterical, blood-covere...",https://nypost.com/2026/01/07/us-news/video-ap...
366,35,2026-01-07-13-07-01 +0000,nypost,Warner Bros. Discovery rejects Paramount Skyda...,https://nypost.com/2026/01/07/business/warner-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
